In [33]:
# Numerical Operations
import math
import numpy as np
 
# Reading/Writing Data
import pandas as pd
import os
import csv
 
# For Progress Bar
from tqdm import tqdm
 
# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
 
# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# DATA

In [43]:
train_data = pd.read_csv('./covid.train_new.csv').values
test_data = pd.read_csv('./covid.test_un.csv').values

In [44]:
train_features = train_data[:2500,1:-1]
train_labels = train_data[:2500,-1]
test_features = test_data[:,1:]
val_features = train_data[2500:,1:-1]
val_labels = train_data[2500:,-1]
print(train_features.shape,train_labels.shape,test_features.shape,val_features.shape,val_labels.shape)

(2500, 116) (2500,) (1078, 116) (199, 116) (199,)


In [45]:
class COVIDataset(Dataset):
    def __init__(self,x,y=None):
        if y is  None:
            self.y = None
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
    
    def __getitem__(self,idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx],self.y[idx]
    
    def __len__(self):
        return len(self.x)

In [46]:
train_dataset = COVIDataset(train_features,train_labels)
val_dataset = COVIDataset(val_features,val_labels)
test_dataset = COVIDataset(test_features)

train_loader = DataLoader(test_dataset,batch_size=64,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=64,shuffle=False)
test_loader = DataLoader(test_data,shuffle=False)

In [47]:
class MyModel(nn.Module):
    def __init__(self,input_dim):
        super(MyModel,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim,64),
            nn.ReLU(),
            nn.Linear(64,8),
            nn.ReLU(),
            nn.Linear(8,1)
        )
    def forward(self,x):
        x = self.layers(x)
        x = squeeze(1)
        return x

In [48]:
model = MyModel(116)
model

MyModel(
  (layers): Sequential(
    (0): Linear(in_features=116, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [49]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-5,momentum=0.9)
model_path = './MyModel.ckpt'
num_epoch = 400

In [50]:
def trainer(model,train_loader,val_loader):
    for epochs in range(400):
        train_loss = []
        val_loss = []
        score = 1e5
        for x,y in train_loader:
            optimizer.zero_grad()
            y_hat = model(x)
            loss = criterion(y_hat,y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss)

        with torch.no_grad():
            for x,y in val_loader:
                y_hat = model(x)
                loss = criterion(y_hat,y)
                val_loss.append(loss)
        print('epoch:',epochs,'train_loss:',sum(train_loss)/len(train_loss),'valid_loss:',sum(val_loss)/len(val_loss),'\n')

In [52]:
trainer(model,train_loader,val_loader)

ValueError: too many values to unpack (expected 2)